# Imports and Variables

In [518]:
from xml.dom import minidom
import io
from PIL import Image  # https://pillow.readthedocs.io/en/4.3.x/
import requests  # http://docs.python-requests.org/en/master/
import re
from pathlib import Path
import os
import pandas as pd


s = 'Residential / Home'
s.split(" / ")
k = 0
pais = estado = cidade = bairro = endereco = cep = ref = titulo = aluguelVenda = 0
tipo_prop = descricao = area = quartos = banheiros = suites = garagem = 0
location_list = []
detail_list = []
images_list = []
paused_list = []
p = s = c = n = a = 0

# Functions

In [519]:
# example image url: https://m.media-amazon.com/images/S/aplus-media/vc/6a9569ab-cb8e-46d9-8aea-a7022e58c74a.jpg
def download_image(url, image_file_path):
    r = requests.get(url, timeout=4.0)
    if r.status_code != requests.codes.ok:
        assert False, 'Status code error: {}.'.format(r.status_code)

    with Image.open(io.BytesIO(r.content)) as im:
        im.save(image_file_path)
    print(f"Image downloaded from url: {format} and saved to: {image_file_path}")


def en_to_pt_resCom(word):
    word = word.split(" / ")
    # Translate the First Word
    if word[0] == "Residential":
        word[0] = 'Residencial'
    else:
        word[0] = 'Comercial'

    # Translate the second Word
    if word[1] == 'Home':
        word[1] = 'Casa'

    elif word[1] == 'Land Lot':
        word[1] = 'Terreno'

    elif word[1] == 'Farm Ranch':
        word[1] = 'Chácaras'

    elif word[1] == 'Apartment':
        word[1] = 'Apartamento'

    else:
        word[1] = word[1]

    return " / ".join(word)


def en_to_pt_rentSale(word):
    if word == 'For Rent':
        word = 'Aluguel'
    else:
        word = 'Venda'
    return word


def format_dir_name(name):
    return name.replace(":","")


def format_nan_null(argument):
    if argument == float("NaN") or argument == 0:
        return '--------'
    else:
        return argument

# Opening the XmL documment

In [520]:
with open("C:/Users/gabri/PycharmProjects/math/arquivos/XML-IMO.xml", 'r', encoding='utf-8') as f:
    xml = minidom.parse(f)
    Listing = xml.getElementsByTagName("Listing")

# First Loop To Find the Location INFO

In [521]:
    for count, tag in enumerate(Listing):
        for count_inside, local in enumerate(tag.getElementsByTagName("Location")):
            if not count_inside:
                ref = (f"referencia: {tag.getElementsByTagName('ListingID')[0].firstChild.data}")
                titulo = (f"Titulo/descricao: {tag.getElementsByTagName('Title')[0].firstChild.data}")
                aluguelVenda = (f"Venda/Aluguel: {en_to_pt_rentSale(tag.getElementsByTagName('TransactionType')[0].firstChild.data)}")
                pais = (f"Pais: {local.getElementsByTagName('Country')[0].firstChild.data}")
                endereco = (f"Endereco/Rua: {local.getElementsByTagName('Address')[0].firstChild.data}")
                estado = (f"Estado: {local.getElementsByTagName('State')[0].firstChild.data}")
                cidade = (f"Cidade: {local.getElementsByTagName('City')[0].firstChild.data}")
                bairro = (f"Bairro: {local.getElementsByTagName('Neighborhood')[0].firstChild.data}")
                try:
                    cep = (f"CEP: {local.getElementsByTagName('PostalCode')[0].firstChild.data}")
                except AttributeError:
                    cep = (f"CEP: --------")
                location_list.append([ref, titulo, cep, endereco, bairro, cidade, estado, pais, aluguelVenda])

# Second loop to find the Listing DETAILS

In [522]:
    for count, tag in enumerate(Listing):
        for count_detail, detail in enumerate(tag.getElementsByTagName("Details")): #ok
            tipo_prop = (f"Finalidade/Imovel: {en_to_pt_resCom(detail.getElementsByTagName('PropertyType')[0].firstChild.data)}")
            descricao = (f"Descricao: {re.sub(' +', ' ', detail.getElementsByTagName('Description')[0].firstChild.wholeText).replace('n', '')}") # alert
            area = (f"Area: {detail.getElementsByTagName('ConstructedArea')[0].firstChild.data}") # alert
            quartos = (f"Quartos: {detail.getElementsByTagName('Bedrooms')[0].firstChild.data}") # alert
            banheiros = (f"Banheiros: {detail.getElementsByTagName('Bathrooms')[0].firstChild.data}") # alert
            suites = (f"Suites: {detail.getElementsByTagName('Suites')[0].firstChild.data}") # alert
            try:
                garagem = (f"Vagas na Garegem: {detail.getElementsByTagName('Garage')[0].firstChild.data}")
            except (AttributeError, IndexError):
                garagem = ("Não tem garagem")

            detail_list.append([descricao, tipo_prop, quartos, suites, banheiros, garagem, area])

# Third Loop to get the images link into a list

In [523]:
    for count, tag in enumerate(Listing):
        for count_detail, image in enumerate(tag.getElementsByTagName("Media")): #ok
            images_list.append([])
            for count_3, links in enumerate(tag.getElementsByTagName("Item")):
                images_list[count].append((links.firstChild.data).replace("\n","").replace(" ", ""))

# Creating Folders to each listing with a description File

In [524]:
# root_path = 'C:/Users/gabri/OneDrive/Pictures/PROGRAMACAO/znuncios'
# for count, items in enumerate(location_list):
#     path = os.path.join(root_path, format_dir_name(items[0]))
#     if not os.path.exists(path):
#         os.makedirs(path)
#     with open(os.path.join(path, format_dir_name(items[0] + '.txt')), 'wt') as temp_file:
#         temp_file.writelines([items[0] + '\n',
#                                 items[1] + '\n',
#                                 items[2] + '\n',
#                                 items[3] + '\n',
#                                 items[4] + '\n',
#                                 items[5] + '\n',
#                                 items[6] + '\n',
#                                 items[7] + '\n',
#                                 items[8] + '\n',
#                                 detail_list[count][0],
#                                 items[1] + '\n',
#                                 items[2] + '\n',
#                                 items[3] + '\n',
#                                 items[4] + '\n',
#                                 items[5] + '\n',
#                                 items[6] + '\n',
#                                 '\n',
#                                 ])
#     print(path)
#     for count_inside, link in enumerate(images_list[count]):
#         download_image(link, os.path.join(path + f"/imagem {count_inside}.jpg"))

In [525]:
# for item in detail_list:
#     print('\n' ,item)

In [526]:
# for item in location_list:
#     print('\n' ,item)

In [527]:
# for item in images_list:
#     print('\n' ,item)

# Reading the Paused listening Xlsx file

In [528]:
xlsx_file = pd.read_excel(r"C:\Users\gabri\PycharmProjects\math\arquivos\AnunciosPausados.xlsx")
for count, line in enumerate(xlsx_file['Ref.']):
    print('\n\n', format_nan_null(xlsx_file['Ref.'][count]))
    print(format_nan_null(xlsx_file['TextoPromocional'][count]))
    print(format_nan_null(xlsx_file['CEP'][count]))
    print(format_nan_null(xlsx_file['Endereco'][count]))
    print(format_nan_null(xlsx_file['Bairro'][count]))
    print(format_nan_null(xlsx_file['Cidade'][count]))
    print(format_nan_null(xlsx_file['UF'][count]))
    print('Brazil')
    print(format_nan_null(xlsx_file['TipoVenda'][count]))
    print(format_nan_null(xlsx_file['Cateogria'][count]))
    print(format_nan_null(xlsx_file['Dormitorio'][count]))
    print(format_nan_null(xlsx_file['Suites'][count]))
    print(format_nan_null(xlsx_file['WC'][count]))
    print(format_nan_null(xlsx_file['VagasAuto'][count]))
    print(format_nan_null(xlsx_file['VagasAuto'][count]))
    print(format_nan_null(xlsx_file['AreaTotal'][count]))
    print(format_nan_null(xlsx_file['AreaTotal'][count]))
    print(format_nan_null(xlsx_file['AreaTotal'][count]))
    print(format_nan_null(xlsx_file['DataCadastro'][count]))
    print(format_nan_null(xlsx_file['MarcadoComoVendido'][count]))
    print(format_nan_null(xlsx_file['OBSInterna'][count]))
    print(format_nan_null(xlsx_file['Proprietario'][count]), (xlsx_file['TelProprietario'][count]))



# ref, titulo, cep, endereco, bairro, cidade, estado, pais, aluguelVenda, descricao, tipo_prop, quartos, suites, banheiros, garagem, area




 247.0
*Galpão   *A se aptar conforme a atividade  *R$ 8,
<class 'float'>
Jose Luiz Leme Maciel, 260
Jordanésia
Jordanesia
SP
Brazil
Aluguel
Salões Comerciais
--------
--------
--------
--------
--------
--------
--------
--------
2012-08-14 14:18:57.873
Não
nan
Cirilo 99945-3845


 493.0
CASA   02 COMODOS, BANHEIRO, LAVANDERIA E GARAGEM.
<class 'float'>
Rua  barbacena, 1007
Jardim Alegria
Francisco Morato
SP
Brazil
Aluguel
Casas
1
--------
1
1
1
--------
--------
--------
2014-03-03 11:53:15.720
Não
nan
Joeme 4091-4878 / 9-6578-8115


 987.0
CASA COM 2 CÔMODOS!
<class 'float'>
nan
Centro
Franco da Rocha
SP
Brazil
Aluguel
Casas
1
--------
1
--------
--------
--------
--------
--------
2017-12-07 10:54:33.810
Sim
CHAVES NA IMOBILIARIA
CLEUSA nan


 888.0
Casa com 3 cômodos.
<class 'float'>
RUA ARCO IRIS 
Lago Azul Ortiz
Franco da Rocha
SP
Brazil
Aluguel
Casas
1
--------
--------
--------
--------
--------
--------
--------
2017-05-02 08:48:04.200
Sim
CHAVES COM PROPRIETÁRIA.
SUELI DE 

In [529]:
# 1378